In [2]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

Read the data

In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_hdf('data/car.h5')

Run Dummy Model

In [4]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [0]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

In [6]:
model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [7]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [8]:
df.price_currency.value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[ df.price_currency != 'EUR' ]

In [10]:
df.price_currency.value_counts()

PLN    106290
Name: price_currency, dtype: int64

Features

In [0]:
suffix = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list):
    continue
  factorized_vals = df[feat].factorize()[0]
  if suffix in feats:
    df[feat] = factorized_vals
  else:
    df[feat + suffix] = factorized_vals

In [0]:
cat_feats = [x for x in df.columns if suffix in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

In [17]:
X.shape, y.shape

((106290, 151), (106290,))

In [18]:
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

np.mean(scores)

-19566.588937368324

In [21]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=2020).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2519 ± 0.0035,param_napęd__cat
0.1995 ± 0.0057,param_faktura-vat__cat
0.1935 ± 0.0059,param_stan__cat
0.1428 ± 0.0066,param_rok-produkcji__cat
0.0634 ± 0.0032,param_moc__cat
0.0423 ± 0.0022,param_skrzynia-biegów__cat
0.0418 ± 0.0017,feature_kamera-cofania__cat
0.0285 ± 0.0046,param_marka-pojazdu__cat
0.0197 ± 0.0044,param_pojemność-skokowa__cat
0.0164 ± 0.0004,feature_bluetooth__cat


In [0]:
!git config --global user.email "m.zajac1988@gmail.com"
!git config --global user.name "Mateusz"

In [0]:
!git add matrix_two/day3_simple_model.ipynb

In [0]:
!git commit -m 'Run simple'